In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import matplotlib 
matplotlib.use('Agg') 
% matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import numpy
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

class IMAGE_GRID(object):
    def __init__(self, images, grid_title='', columns=5):
        rows = max(int(math.ceil(len(images) / float(columns))), 1)
        self._init_grid_plot(columns, rows)
        plt.suptitle(grid_title, fontsize=30)
        for position, image in enumerate(images):
            self._show_one_image(image, position)
        
    def _init_grid_plot(self, columns, rows, factor=1.0):
        matplotlib.rcParams.update({'font.size': 18})
        matplotlib.rcParams.update({'figure.figsize':(round(columns * 5 / float(factor)),
                                                round(rows * 4 / float(factor)))})
        self.plot_grid = gridspec.GridSpec(rows, columns)
        self.plot_grid.update(wspace=0.01, hspace=0.05)  # set the spacing between axes.

    def _show_one_image(self, image, plot_id):
        ax = plt.subplot(self.plot_grid[plot_id])
        plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
        plt.tick_params(axis='y', which='both', bottom='off', left='off', labelleft='off')
        plt.imshow(image)

In [4]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,exception_verbosity=high"

In [5]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('./UECFOOD100/cropped/1/53_0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
batches = list()
for batch in datagen.flow(x, batch_size=1):
    i += 1
    batches.append(batch)
    if i > 20:
        break  # otherwise the generator would loop indefinitely

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [ ]:
IMAGE_GRID([array_to_img(entry[0]) for entry in batches], columns=5)

In [35]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


# dimensions of our images.
img_width, img_height = 150, 150



model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_width, img_height)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [37]:
train_data_dir = './UECFOOD100/data/train'
validation_data_dir = './UECFOOD100/data/validation'
nb_train_samples = 10183
nb_validation_samples = 4428
nb_epoch = 50
batch_size = 64
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

# model.save_weights('first_try.h5')

Found 10183 images belonging to 100 classes.
Found 4428 images belonging to 100 classes.
Epoch 1/50
10183/10183 [==============================] - 120s - loss: 4.2983 - acc: 0.0867 - val_loss: 4.0873 - val_acc: 0.1107
Epoch 2/50
10183/10183 [==============================] - 120s - loss: 4.0796 - acc: 0.1040 - val_loss: 3.8698 - val_acc: 0.1375
Epoch 3/50
10183/10183 [==============================] - 118s - loss: 3.9036 - acc: 0.1281 - val_loss: 3.7226 - val_acc: 0.1603
Epoch 4/50
10183/10183 [==============================] - 127s - loss: 3.7655 - acc: 0.1436 - val_loss: 3.7794 - val_acc: 0.1416
Epoch 5/50
10183/10183 [==============================] - 121s - loss: 3.6790 - acc: 0.1650 - val_loss: 3.3878 - val_acc: 0.2107
Epoch 6/50
10183/10183 [==============================] - 120s - loss: 3.5788 - acc: 0.1767 - val_loss: 3.6286 - val_acc: 0.1682
Epoch 7/50
10183/10183 [==============================] - 118s - loss: 3.5213 - acc: 0.1863 - val_loss: 3.4798 - val_acc: 0.1906
Epoch 8/

KeyboardInterrupt: 

In [25]:
# model.load_weights('second_try.h5')

In [38]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_10 (Convolution2D) (None, 32, 148, 148)  896         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
activation_16 (Activation)       (None, 32, 148, 148)  0           convolution2d_10[0][0]           
____________________________________________________________________________________________________
maxpooling2d_10 (MaxPooling2D)   (None, 32, 74, 74)    0           activation_16[0][0]              
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 32, 72, 72)    9248        maxpooling2d_10[0][0]            
___________________________________________________________________________________________

In [27]:
model.predict(x)

Exception: Error when checking : expected convolution2d_input_1 to have shape (None, 3, 150, 150) but got array with shape (1, 3, 425, 432)

In [28]:
x = img_to_array(img)
x = x.reshape((1,) + x.shape)


model.predict_classes(x)


Exception: Error when checking : expected convolution2d_input_1 to have shape (None, 3, 150, 150) but got array with shape (1, 3, 425, 432)

In [40]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = './UECFOOD100/data/train'
validation_data_dir = './UECFOOD100/data/validation'
nb_train_samples = 10183
nb_validation_samples = 4428
nb_epoch = 50


model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_width, img_height)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=128,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=128,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

# model.save_weights('second_try.h5')

Found 10183 images belonging to 100 classes.
Found 4428 images belonging to 100 classes.
Epoch 1/50
10183/10183 [==============================] - 128s - loss: 4.5418 - acc: 0.0628 - val_loss: 4.3717 - val_acc: 0.0779
Epoch 2/50
10183/10183 [==============================] - 125s - loss: 4.3470 - acc: 0.0796 - val_loss: 4.0970 - val_acc: 0.0874
Epoch 3/50
10183/10183 [==============================] - 128s - loss: 4.1637 - acc: 0.0943 - val_loss: 3.9863 - val_acc: 0.1360
Epoch 4/50
10183/10183 [==============================] - 124s - loss: 4.0100 - acc: 0.1133 - val_loss: 3.7609 - val_acc: 0.1454
Epoch 5/50
10183/10183 [==============================] - 124s - loss: 3.8828 - acc: 0.1322 - val_loss: 3.7831 - val_acc: 0.1425
Epoch 6/50
10183/10183 [==============================] - 124s - loss: 3.7895 - acc: 0.1442 - val_loss: 3.5912 - val_acc: 0.1721
Epoch 7/50
10183/10183 [==============================] - 125s - loss: 3.6930 - acc: 0.1609 - val_loss: 3.6417 - val_acc: 0.1757
Epoch 8/

In [53]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = './UECFOOD100/data/train'
validation_data_dir = './UECFOOD100/data/validation'
nb_train_samples = 10183
nb_validation_samples = 4428
nb_epoch = 50


model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, img_width, img_height)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

model.save_weights('third_try.h5')

Found 10183 images belonging to 100 classes.
Found 4428 images belonging to 100 classes.
Epoch 1/50
10183/10183 [==============================] - 153s - loss: 4.3982 - acc: 0.0736 - val_loss: 4.3785 - val_acc: 0.0858
Epoch 2/50
10183/10183 [==============================] - 150s - loss: 4.1397 - acc: 0.1083 - val_loss: 4.5767 - val_acc: 0.0829
Epoch 3/50
10183/10183 [==============================] - 148s - loss: 3.9528 - acc: 0.1242 - val_loss: 3.9761 - val_acc: 0.1253
Epoch 4/50
10183/10183 [==============================] - 149s - loss: 3.8379 - acc: 0.1406 - val_loss: 3.6554 - val_acc: 0.1703
Epoch 5/50
10183/10183 [==============================] - 148s - loss: 3.7076 - acc: 0.1579 - val_loss: 3.6474 - val_acc: 0.1789
Epoch 6/50
 9792/10183 [===========================>..] - ETA: 4s - loss: 3.6181 - acc: 0.1724

KeyboardInterrupt: 

In [39]:
import h5py
import numpy as np
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D

# path to the model weights file.
weights_path = 'D:/DSR/neural-networks-and-deep-learning/CatsandDogs/models/vgg16_weights.h5'
top_model_weights_path = './bottleneck_fc_model.h5'
# dimensions of our images.

nb_train_samples = 10183
nb_validation_samples = 4428

train_data_dir = './UECFOOD100/data/train'
validation_data_dir = './UECFOOD100/data/validation'


datagen = ImageDataGenerator(rescale=1./255)
#datagen = ImageDataGenerator(rescale=1., featurewise_center=True) 
#datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(3,1,1)

# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)
bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
###NONONONONONO## np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)
# np.save(open('bottleneck_features_train.npy', 'ba'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
######NONONONONONO#np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)
# np.save(open('bottleneck_features_validation.npy', 'ba'), bottleneck_features_validation)


Model loaded.
Found 10183 images belonging to 100 classes.


KeyboardInterrupt: 

In [47]:
generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

Found 10183 images belonging to 100 classes.


In [24]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

img_width, img_height = 150,150
nb_train_samples = 10183


train_data_dir = './UECFOOD100/data/train'

datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)
i=0
train_labels = np.empty((0,100))
for _, yy  in generator:
    i += 1
    train_labels = np.vstack([train_labels,yy[0]])
    if i >= nb_train_samples:
        break

Found 10183 images belonging to 100 classes.


In [10]:
import pandas as pd
dftrain = pd.DataFrame(generator.filenames)

In [29]:
dftrain

,0
0,1\10572_0.jpg
1,1\10586_0.jpg
2,1\10617_0.jpg
3,1\10618_0.jpg
4,1\10621_0.jpg
5,1\10630_0.jpg
6,1\10633_0.jpg
7,1\10634_0.jpg
8,1\10640_0.jpg
9,1\10666_0.jpg


In [51]:
dftrain['labels'] = dftrain.apply(lambda x: str(x).strip().split('\\')[0], axis=1)

In [49]:
dftrain

,0,labels
0,1\10572_0.jpg,"10572_0.jpg\nlabels 0 1\nName: 0,..."
1,1\10586_0.jpg,"10586_0.jpg\nlabels 0 1\nName: 1,..."
2,1\10617_0.jpg,"10617_0.jpg\nlabels 0 1\nName: 2,..."
3,1\10618_0.jpg,"10618_0.jpg\nlabels 0 1\nName: 3,..."
4,1\10621_0.jpg,"10621_0.jpg\nlabels 0 1\nName: 4,..."
5,1\10630_0.jpg,"10630_0.jpg\nlabels 0 1\nName: 5,..."
6,1\10633_0.jpg,"10633_0.jpg\nlabels 0 1\nName: 6,..."
7,1\10634_0.jpg,"10634_0.jpg\nlabels 0 1\nName: 7,..."
8,1\10640_0.jpg,"10640_0.jpg\nlabels 0 1\nName: 8,..."
9,1\10666_0.jpg,"10666_0.jpg\nlabels 0 1\nName: 9,..."


In [48]:
str(dftrain[4]).strip().split('\\')

KeyError: 4

In [50]:
dftrain

,0,labels
0,1\10572_0.jpg,"10572_0.jpg\nlabels 0 1\nName: 0,..."
1,1\10586_0.jpg,"10586_0.jpg\nlabels 0 1\nName: 1,..."
2,1\10617_0.jpg,"10617_0.jpg\nlabels 0 1\nName: 2,..."
3,1\10618_0.jpg,"10618_0.jpg\nlabels 0 1\nName: 3,..."
4,1\10621_0.jpg,"10621_0.jpg\nlabels 0 1\nName: 4,..."
5,1\10630_0.jpg,"10630_0.jpg\nlabels 0 1\nName: 5,..."
6,1\10633_0.jpg,"10633_0.jpg\nlabels 0 1\nName: 6,..."
7,1\10634_0.jpg,"10634_0.jpg\nlabels 0 1\nName: 7,..."
8,1\10640_0.jpg,"10640_0.jpg\nlabels 0 1\nName: 8,..."
9,1\10666_0.jpg,"10666_0.jpg\nlabels 0 1\nName: 9,..."


In [121]:
gen[1].shape

(1, 100)

In [9]:
validation_data_dir = './UECFOOD100/data/validation'
import numpy as np

img_width, img_height = 150,150
nb_validation_samples = 4428

generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)
i=0
validation_labels= np.empty((0,100))
for _, yy in generator:
    i += 1
#     x,yy = batch
    validation_labels = np.vstack([validation_labels,yy])
    if i >= nb_validation_samples:
        break

Found 4428 images belonging to 100 classes.


In [17]:
# generator.class_indices
validation_labels.argmax(axis=-1)[188:500]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8,

In [18]:
generator.class_indices['10']

1

In [20]:
from keras.utils import np_utils
np_utils.to_categorical(99, 100)

TypeError: object of type 'int' has no len()

In [14]:
type(validation_labels[[1,500],:])

numpy.ndarray

In [114]:
validation_labels[0,:].shape

(100,)

In [101]:
train_labels.shape

(10183, 100)

In [115]:
train_labels[0,:].shape

(100,)

In [122]:
print (bottleneck_features_train.shape)

NameError: name 'bottleneck_features_train' is not defined

In [38]:
from keras.optimizers import SGD
from keras.layers import SpatialDropout2D

sgd = SGD(lr=0.01, clipnorm=1.)
train_data = np.load(open('bottleneck_features_train.npy','rb'))

#train_labels = np.array([0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))
# train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

# validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))

# validation_labels = np.array([0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))
# validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(100, activation='sigmoid'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          nb_epoch=nb_epoch, 
          batch_size=32,
          validation_data=(validation_data, validation_labels))

# model.save_weights(top_model_weights_path)
# model.save_weights('./bottleneck_fc_model_24_11_16.h5')

Train on 10183 samples, validate on 4428 samples
Epoch 1/50
10183/10183 [==============================] - 2s - loss: 4.5366 - acc: 0.0497 - val_loss: 4.4542 - val_acc: 0.0639
Epoch 2/50
10183/10183 [==============================] - 2s - loss: 4.4047 - acc: 0.0646 - val_loss: 4.3037 - val_acc: 0.0903
Epoch 3/50
10183/10183 [==============================] - 2s - loss: 4.2729 - acc: 0.0775 - val_loss: 4.0938 - val_acc: 0.0804
Epoch 4/50
10183/10183 [==============================] - 2s - loss: 4.1170 - acc: 0.0859 - val_loss: 3.9129 - val_acc: 0.1028
Epoch 5/50
10183/10183 [==============================] - 2s - loss: 3.9559 - acc: 0.0959 - val_loss: 3.7231 - val_acc: 0.0901
Epoch 6/50
10183/10183 [==============================] - 2s - loss: 3.7936 - acc: 0.1195 - val_loss: 3.4500 - val_acc: 0.1384
Epoch 7/50
10183/10183 [==============================] - 2s - loss: 3.6314 - acc: 0.1448 - val_loss: 3.4080 - val_acc: 0.1902
Epoch 8/50
10183/10183 [==============================] - 2s -

In [ ]:
model.predict_classes()

In [43]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

img_width, img_height = 150, 150

weights_path = 'D:/DSR/neural-networks-and-deep-learning/CatsandDogs/models/vgg16_weights.h5'
top_model_weights_path = './bottleneck_fc_model.h5'

model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')
model.output_shape[1:]


Model loaded.


(512, 4, 4)

In [44]:
model.output_shape

(None, 512, 4, 4)

In [ ]:
train_data_dir = './UECFOOD100/data/train'
validation_data_dir = './UECFOOD100/data/validation'
nb_train_samples = 10183
nb_validation_samples = 4428
nb_epoch = 50

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))

top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(100, activation='sigmoid'))



# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights('./bottleneck_fc_model_24_11_16.h5')

# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
#               optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

In [22]:
top_model.output_shape

(None, 100)

In [82]:
# from keras.preprocessing import image
# # from imagenet_utils import preprocess_input
# from keras.models import Model

# # base_model = VGG19(weights='imagenet')
# # model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)

# img_path = './UECFOOD100/cropped/1/53_0.jpg'
# # img = load_img('./UECFOOD100/cropped/1/53_0.jpg')  # this is a PIL image

# img = image.load_img(img_path, target_size=(150, 150))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# # x = preprocess_input(x)

# model.predict(x)